### Regression Analysis

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import statsmodels.api as sm

In [ ]:
data = pd.read_csv("~/Pricing_analytics/cereal_merged_df.csv")
data

,Unnamed: 0,WEEK_END_DATE,STORE_NUM,UPC,UNITS,VISITS,HHS,SPEND,PRICE,BASE_PRICE,...,PRODUCT_SIZE,STORE_ID,STORE_NAME,ADDRESS_CITY_NAME,ADDRESS_STATE_PROV_CODE,MSA_CODE,SEG_VALUE_NAME,PARKING_SPACE_QTY,SALES_AREA_SIZE_NUM,AVG_WEEKLY_BASKETS
0,0,2009-01-14,367.0,1111085319,14.0,13.0,13.0,26.32,1.88,1.88,...,12.25 OZ,367,15TH & MADISON,COVINGTON,KY,17140,VALUE,196.0,24721,12706.532051
1,1,2009-01-21,367.0,1111085319,12.0,12.0,12.0,22.68,1.89,1.89,...,12.25 OZ,367,15TH & MADISON,COVINGTON,KY,17140,VALUE,196.0,24721,12706.532051
2,2,2009-01-28,367.0,1111085319,18.0,17.0,16.0,33.66,1.87,1.87,...,12.25 OZ,367,15TH & MADISON,COVINGTON,KY,17140,VALUE,196.0,24721,12706.532051
3,3,2009-02-04,367.0,1111085319,13.0,13.0,13.0,24.44,1.88,1.88,...,12.25 OZ,367,15TH & MADISON,COVINGTON,KY,17140,VALUE,196.0,24721,12706.532051
4,4,2009-02-11,367.0,1111085319,16.0,16.0,16.0,29.92,1.87,1.87,...,12.25 OZ,367,15TH & MADISON,COVINGTON,KY,17140,VALUE,196.0,24721,12706.532051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169684,169684,2011-12-07,29159.0,88491212971,12.0,12.0,12.0,34.20,2.85,2.85,...,11 OZ,29159,CARROLLTON,CARROLLTON,TX,19100,MAINSTREAM,NaN,54927,25916.532051
169685,169685,2011-12-14,29159.0,88491212971,14.0,14.0,14.0,39.90,2.85,2.85,...,11 OZ,29159,CARROLLTON,CARROLLTON,TX,19100,MAINSTREAM,NaN,54927,25916.532051
169686,169686,2011-12-21,29159.0,88491212971,22.0,22.0,21.0,62.70,2.85,2.85,...,11 OZ,29159,CARROLLTON,CARROLLTON,TX,19100,MAINSTREAM,NaN,54927,25916.532051
169687,169687,2011-12-28,29159.0,88491212971,17.0,16.0,16.0,48.45,2.85,2.85,...,11 OZ,29159,CARROLLTON,CARROLLTON,TX,19100,MAINSTREAM,NaN,54927,25916.532051


### Organize DataFrames for the analysis

In [ ]:
data.MANUFACTURER.value_counts()

GENERAL MI       35948
PRIVATE LABEL    35902
KELLOGG          35832
POST FOODS       31107
QUAKER           30900
Name: MANUFACTURER, dtype: int64

In [ ]:
data.SUB_CATEGORY.value_counts()

ALL FAMILY CEREAL    82006
KIDS CEREAL          54690
ADULT CEREAL         32993
Name: SUB_CATEGORY, dtype: int64

In [ ]:
# create 2 DataFrames based on SUB_CATEGORY
family = data.loc[data['SUB_CATEGORY']=='ALL FAMILY CEREAL']
kids= data.loc[data['SUB_CATEGORY']=='KIDS CEREAL']

# store original mean price 
family_price = family.PRICE.mean()
kids_price = kids.PRICE.mean()

print('Family - mean price is:', family_price)
print('Kids - mean price is:', kids_price)

Family - mean price is: 2.8015646416115896
Kids - mean price is: 2.6833255924517267


In [ ]:
# for preparing data for regression 
def data_prep(data):
    # kellog
    df = data.loc[data['MANUFACTURER']=="KELLOGG"].groupby(['WEEK_END_DATE','STORE_NUM']).agg({'PRICE':'mean','UNITS':'sum'}).reset_index()
    df.rename(columns={"PRICE":'kellog_price','UNITS':'kellog_units'},inplace=True)
    
    # other brands
    mi = data.loc[data['MANUFACTURER']=="GENERAL MI"].groupby(['WEEK_END_DATE','STORE_NUM'])['PRICE'].mean().reset_index()['PRICE']
    private = data.loc[data['MANUFACTURER']=="PRIVATE LABEL"].groupby(['WEEK_END_DATE','STORE_NUM'])['PRICE'].mean().reset_index()['PRICE']
    post = data.loc[data['MANUFACTURER']=="POST FOODS"].groupby(['WEEK_END_DATE','STORE_NUM'])['PRICE'].mean().reset_index()['PRICE']
    quaker = data.loc[data['MANUFACTURER']=="QUAKER"].groupby(['WEEK_END_DATE','STORE_NUM'])['PRICE'].mean().reset_index()['PRICE']

    df['mi_price'] = mi
    df['private_price'] = private
    df['post_price'] = post
    df['quaker_price'] = quaker
    
    # check # of transactions for each brand 
    print('kellog:',df.shape[0])
    print('General MI:',mi.shape[0])
    print('Private Label:',private.shape[0])
    print('Post Foods:',post.shape[0])
    print('Quarker:',quaker.shape[0])
    
    # convert prices variables to log-prices
    to_log = ['kellog_units','kellog_price','mi_price','private_price','post_price','quaker_price']
    df[to_log] = np.log(df[to_log])
    
    # specify categorical variables
    df['STORE_NUM'] = df['STORE_NUM'].astype('category')
    
    return (df)

### Family

In [ ]:
df = data_prep(family) # post doesn't do Family cereal

kellog: 11904
General MI: 11987
Private Label: 11986
Post Foods: 0
Quarker: 10232


In [ ]:
# get rid of rows with NaN
df = df.loc[df['quaker_price'].notnull()]
df.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10232 entries, 0 to 10231
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   WEEK_END_DATE  10232 non-null  object  
 1   STORE_NUM      10232 non-null  category
 2   kellog_price   10232 non-null  float64 
 3   kellog_units   10232 non-null  float64 
 4   mi_price       10232 non-null  float64 
 5   private_price  10232 non-null  float64 
 6   post_price     0 non-null      float64 
 7   quaker_price   10232 non-null  float64 
dtypes: category(1), float64(6), object(1)
memory usage: 652.1+ KB


#### Model 1: only price variables 

In [ ]:
Y=df['kellog_units']
X=df[['kellog_price','mi_price','private_price','quaker_price']]
X = sm.add_constant(X) # adding a constant

model_1 = sm.OLS(Y,X).fit()
model_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           kellog_units   R-squared:                       0.079
Model:                            OLS   Adj. R-squared:                  0.079
Method:                 Least Squares   F-statistic:                     219.0
Date:                Thu, 03 Jun 2021   Prob (F-statistic):          1.20e-180
Time:                        05:04:46   Log-Likelihood:                -11811.
No. Observations:               10232   AIC:                         2.363e+04
Df Residuals:                   10227   BIC:                         2.367e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             4.1087      0.144     28.589      0.000       3.827       4.390
kellog_price     -1.7979      0.064    -27.996      0.000      -1.924      -1.672
mi_price          0.1872      0.071      2.637      0.008       0.048       0.326
private_price     1.1652      0.128      9.108      0.000       0.914       1.416
quaker_price      0.1686      0.039      4.349      0.000       0.093       0.245
==============================================================================
Omnibus:                      961.732   Durbin-Watson:                   1.679
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1428.985
Skew:                          -0.726   Prob(JB):                    5.01e-311
Kurtosis:                       4.115   Cond. No.                         51.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
# Optimal price = VC * {e/(1+e)}
price = (family_price*0.6)*(-1.7979/(1-1.7979))
price

3.787642363068161

#### Model 2: price variables + store_num

In [ ]:
# dummy variable
df = pd.get_dummies(data=df,columns=['STORE_NUM'], drop_first = True)

In [ ]:
Y=df['kellog_units']
X = df.drop(['WEEK_END_DATE','kellog_units','post_price'],axis=1)
X = sm.add_constant(X) # adding a constant

model_2 = sm.OLS(Y,X).fit()
model_2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           kellog_units   R-squared:                       0.705
Model:                            OLS   Adj. R-squared:                  0.702
Method:                 Least Squares   F-statistic:                     302.8
Date:                Thu, 03 Jun 2021   Prob (F-statistic):               0.00
Time:                        05:05:05   Log-Likelihood:                -5991.3
No. Observations:               10232   AIC:                         1.214e+04
Df Residuals:                   10151   BIC:                         1.273e+04
Df Model:                          80                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 4.8034      0.092     52.093      0.000       4.623       4.984
kellog_price         -2.4146      0.038    -63.047      0.000      -2.490      -2.339
mi_price              0.1862      0.040      4.604      0.000       0.107       0.265
private_price         1.0819      0.074     14.719      0.000       0.938       1.226
quaker_price          0.1341      0.022      6.078      0.000       0.091       0.177
STORE_NUM_387.0       0.6872      0.056     12.240      0.000       0.577       0.797
STORE_NUM_389.0       1.0151      0.053     19.076      0.000       0.911       1.119
STORE_NUM_613.0       0.2280      0.053      4.283      0.000       0.124       0.332
STORE_NUM_623.0      -0.0556      0.053     -1.044      0.296      -0.160       0.049
STORE_NUM_2277.0      1.4151      0.053     26.596      0.000       1.311       1.519
STORE_NUM_2279.0      0.5393      0.053     10.136      0.000       0.435       0.644
STORE_NUM_2281.0      0.9801      0.053     18.420      0.000       0.876       1.084
STORE_NUM_2495.0     -0.0082      0.053     -0.154      0.878      -0.113       0.096
STORE_NUM_2505.0     -0.4246      0.054     -7.935      0.000      -0.530      -0.320
STORE_NUM_2513.0      0.4602      0.053      8.635      0.000       0.356       0.565
STORE_NUM_2523.0     -0.9367      0.053    -17.606      0.000      -1.041      -0.832
STORE_NUM_2541.0     -0.9410      0.053    -17.686      0.000      -1.045      -0.837
STORE_NUM_4245.0      0.7520      0.053     14.125      0.000       0.648       0.856
STORE_NUM_4259.0      0.1169      0.053      2.195      0.028       0.012       0.221
STORE_NUM_4489.0      0.0400      0.053      0.750      0.453      -0.064       0.145
STORE_NUM_4503.0      0.4816      0.053      9.048      0.000       0.377       0.586
STORE_NUM_4521.0     -0.7786      0.053    -14.602      0.000      -0.883      -0.674
STORE_NUM_6179.0      0.6954      0.053     13.068      0.000       0.591       0.800
STORE_NUM_6187.0      0.6495      0.053     12.207      0.000       0.545       0.754
STORE_NUM_6379.0      0.3128      0.053      5.870      0.000       0.208       0.417
STORE_NUM_6431.0     -0.3943      0.053     -7.398      0.000      -0.499      -0.290
STORE_NUM_8035.0     -1.8959      0.067    -28.106      0.000      -2.028      -1.764
STORE_NUM_8041.0      0.1352      0.053      2.537      0.011       0.031       0.240
STORE_NUM_8263.0      0.1953      0.053      3.658      0.000       0.091       0.300
STORE_NUM_9825.0      0.8340      0.053     15.644      0.000       0.730       0.939
STORE_NUM_10019.0    -0.9621      0.053    -18.013      0.000      -1.067      -0.857
STORE_NUM_11757.0     0.2886      0.053      5.413      0.000       0.184       0.393
STORE_NUM_11761.0     0.7432      0.053     13.942      0.000       0.639

In [ ]:
# Optimal price = VC * {e/(1+e)}
price = (family_price*0.6)*(-2.4146/(1-2.4146))
price

2.86921729830426

#### Model 3: price variables + store_num + week_end_date 

In [ ]:
# dummy variable
df = pd.get_dummies(data=df,columns=['WEEK_END_DATE'], drop_first = True)

In [ ]:
Y=df['kellog_units']
X = df.drop(['kellog_units','post_price'],axis=1)
X = sm.add_constant(X) # adding a constant

model_3 = sm.OLS(Y,X).fit()
model_3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           kellog_units   R-squared:                       0.807
Model:                            OLS   Adj. R-squared:                  0.803
Method:                 Least Squares   F-statistic:                     196.3
Date:                Thu, 03 Jun 2021   Prob (F-statistic):               0.00
Time:                        05:05:34   Log-Likelihood:                -3802.2
No. Observations:               10232   AIC:                             8034.
Df Residuals:                   10017   BIC:                             9589.
Df Model:                         214                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                        5.0036      0.132     37.869      0.000       4.745       5.263
kellog_price                -1.9090      0.071    -26.798      0.000      -2.049      -1.769
mi_price                     0.0664      0.061      1.082      0.279      -0.054       0.187
private_price                0.0151      0.094      0.160      0.873      -0.169       0.199
quaker_price                 0.0341      0.027      1.277      0.202      -0.018       0.087
STORE_NUM_387.0              0.7109      0.046     15.554      0.000       0.621       0.800
STORE_NUM_389.0              1.0039      0.043     23.203      0.000       0.919       1.089
STORE_NUM_613.0              0.2231      0.043      5.156      0.000       0.138       0.308
STORE_NUM_623.0             -0.0163      0.044     -0.373      0.709      -0.102       0.069
STORE_NUM_2277.0             1.4121      0.043     32.650      0.000       1.327       1.497
STORE_NUM_2279.0             0.5420      0.043     12.531      0.000       0.457       0.627
STORE_NUM_2281.0             0.9943      0.043     22.985      0.000       0.910       1.079
STORE_NUM_2495.0             0.0294      0.044      0.675      0.499      -0.056       0.115
STORE_NUM_2505.0            -0.4011      0.044     -9.143      0.000      -0.487      -0.315
STORE_NUM_2513.0             0.4829      0.044     11.078      0.000       0.397       0.568
STORE_NUM_2523.0            -0.9333      0.043    -21.581      0.000      -1.018      -0.849
STORE_NUM_2541.0            -0.9337      0.043    -21.587      0.000      -1.019      -0.849
STORE_NUM_4245.0             0.7268      0.043     16.770      0.000       0.642       0.812
STORE_NUM_4259.0             0.0874      0.043      2.017      0.044       0.002       0.172
STORE_NUM_4489.0             0.0530      0.044      1.214      0.225      -0.033       0.138
STORE_NUM_4503.0             0.4642      0.043     10.722      0.000       0.379       0.549
STORE_NUM_4521.0            -0.7738      0.043    -17.833      0.000      -0.859      -0.689
STORE_NUM_6179.0             0.6892      0.043     15.930      0.000       0.604       0.774
STORE_NUM_6187.0             0.6700      0.043     15.486      0.000       0.585       0.755
STORE_NUM_6379.0             0.3451      0.044      7.920      0.000       0.260       0.431
STORE_NUM_6431.0            -0.3740      0.044     -8.574      0.000      -0.460      -0.289
STORE_NUM_8035.0            -1.9467      0.055    -35.393      0.000      -2.055      -1.839
STORE_NUM_8041.0             0.1345      0.043      3.103      0.002       0.050       0.220
STORE_NUM_8263.0             0.2404      0.044      5.506      0.000       0.155       0.326
STORE_NUM_9825.0             0.8455      0.043     19.499      0.000       0.761       0.931
STORE_NUM_1001

In [ ]:
# Optimal price = VC * {e/(1+e)}
price = (family_price*0.6)*(-1.9090/(1-1.9090))
price

3.5301563701891245

#### Bias induced by not considering competition

In [ ]:
# regression with No competitors
Y=df['kellog_units']
#X=df[['kellog_price']]
X = df.drop(['kellog_units','post_price','mi_price','private_price','quaker_price'],axis=1)
X = sm.add_constant(X) # adding a constant

model_1_b = sm.OLS(Y,X).fit()
model_1_b.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           kellog_units   R-squared:                       0.807
Model:                            OLS   Adj. R-squared:                  0.803
Method:                 Least Squares   F-statistic:                     199.1
Date:                Thu, 03 Jun 2021   Prob (F-statistic):               0.00
Time:                        05:06:10   Log-Likelihood:                -3803.6
No. Observations:               10232   AIC:                             8031.
Df Residuals:                   10020   BIC:                             9565.
Df Model:                         211                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                        5.1165      0.098     52.138      0.000       4.924       5.309
kellog_price                -1.9079      0.071    -26.787      0.000      -2.048      -1.768
STORE_NUM_387.0              0.7100      0.046     15.538      0.000       0.620       0.800
STORE_NUM_389.0              1.0036      0.043     23.203      0.000       0.919       1.088
STORE_NUM_613.0              0.2233      0.043      5.163      0.000       0.139       0.308
STORE_NUM_623.0             -0.0160      0.044     -0.367      0.714      -0.101       0.069
STORE_NUM_2277.0             1.4120      0.043     32.651      0.000       1.327       1.497
STORE_NUM_2279.0             0.5425      0.043     12.544      0.000       0.458       0.627
STORE_NUM_2281.0             0.9947      0.043     22.995      0.000       0.910       1.079
STORE_NUM_2495.0             0.0292      0.044      0.670      0.503      -0.056       0.115
STORE_NUM_2505.0            -0.4015      0.044     -9.161      0.000      -0.487      -0.316
STORE_NUM_2513.0             0.4820      0.044     11.065      0.000       0.397       0.567
STORE_NUM_2523.0            -0.9330      0.043    -21.576      0.000      -1.018      -0.848
STORE_NUM_2541.0            -0.9342      0.043    -21.600      0.000      -1.019      -0.849
STORE_NUM_4245.0             0.7265      0.043     16.775      0.000       0.642       0.811
STORE_NUM_4259.0             0.0867      0.043      2.004      0.045       0.002       0.171
STORE_NUM_4489.0             0.0516      0.044      1.184      0.236      -0.034       0.137
STORE_NUM_4503.0             0.4631      0.043     10.710      0.000       0.378       0.548
STORE_NUM_4521.0            -0.7743      0.043    -17.855      0.000      -0.859      -0.689
STORE_NUM_6179.0             0.6885      0.043     15.922      0.000       0.604       0.773
STORE_NUM_6187.0             0.6705      0.043     15.497      0.000       0.586       0.755
STORE_NUM_6379.0             0.3453      0.044      7.927      0.000       0.260       0.431
STORE_NUM_6431.0            -0.3756      0.044     -8.618      0.000      -0.461      -0.290
STORE_NUM_8035.0            -1.9501      0.055    -35.500      0.000      -2.058      -1.842
STORE_NUM_8041.0             0.1343      0.043      3.099      0.002       0.049       0.219
STORE_NUM_8263.0             0.2398      0.044      5.493      0.000       0.154       0.325
STORE_NUM_9825.0             0.8455      0.043     19.505      0.000       0.761       0.931
STORE_NUM_10019.0           -0.9492      0.044    -21.759      0.000      -1.035      -0.864
STORE_NUM_11757.0            0.2918      0.043      6.732      0.000       0.207       0.377
STORE_NUM_11761.0            0.7410      0.043     17.101      0.000       0.656       0.826
STORE_NUM_1196

In [ ]:
with_comp = price
#no_comp = (family_price*0.6)*(-1.7718/(1-1.7718))
no_comp = (family_price*0.6)*(-1.9079/(1-1.9079))
no_comp

3.5323968585069405

In [ ]:
bias = with_comp - no_comp
percent_bias = 100*(bias/with_comp)
percent_bias

-0.06346711258277629

### Kids

In [ ]:
df = data_prep(kids) # General MI and Private don't do Kids cereal

kellog: 11987
General MI: 0
Private Label: 0
Post Foods: 10094
Quarker: 10588


In [ ]:
# get rid of rows with NaN
df = df.loc[df['post_price'].notnull()]
df.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10094 entries, 0 to 10093
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   WEEK_END_DATE  10094 non-null  object  
 1   STORE_NUM      10094 non-null  category
 2   kellog_price   10094 non-null  float64 
 3   kellog_units   10094 non-null  float64 
 4   mi_price       0 non-null      float64 
 5   private_price  0 non-null      float64 
 6   post_price     10094 non-null  float64 
 7   quaker_price   10094 non-null  float64 
dtypes: category(1), float64(6), object(1)
memory usage: 643.4+ KB


#### Model 1: only price variables 

In [ ]:
Y=df['kellog_units']
X=df[['kellog_price','post_price','quaker_price']]
X = sm.add_constant(X) # adding a constant

model_1 = sm.OLS(Y,X).fit()
model_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           kellog_units   R-squared:                       0.432
Model:                            OLS   Adj. R-squared:                  0.432
Method:                 Least Squares   F-statistic:                     2563.
Date:                Thu, 03 Jun 2021   Prob (F-statistic):               0.00
Time:                        05:08:25   Log-Likelihood:                -9040.4
No. Observations:               10094   AIC:                         1.809e+04
Df Residuals:                   10090   BIC:                         1.812e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            7.9302      0.074    107.494      0.000       7.786       8.075
kellog_price    -4.1487      0.048    -86.435      0.000      -4.243      -4.055
post_price       0.2728      0.039      7.045      0.000       0.197       0.349
quaker_price     0.2849      0.045      6.323      0.000       0.197       0.373
==============================================================================
Omnibus:                      160.092   Durbin-Watson:                   1.358
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              262.475
Skew:                          -0.142   Prob(JB):                     1.01e-57
Kurtosis:                       3.737   Cond. No.                         28.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
# Optimal price = VC * {e/(1+e)}
price = (family_price*0.6)*(-4.1487/(1-4.1487))
price

2.2147904650148953

#### Model 2: price variables + store_num

In [ ]:
# dummy variable
df = pd.get_dummies(data=df,columns=['STORE_NUM'], drop_first = True)

In [ ]:
Y=df['kellog_units']
X = df.drop(['WEEK_END_DATE','kellog_units','mi_price','private_price'],axis=1)
X = sm.add_constant(X) # adding a constant

model_2 = sm.OLS(Y,X).fit()
model_2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           kellog_units   R-squared:                       0.667
Model:                            OLS   Adj. R-squared:                  0.664
Method:                 Least Squares   F-statistic:                     253.8
Date:                Thu, 03 Jun 2021   Prob (F-statistic):               0.00
Time:                        05:09:19   Log-Likelihood:                -6350.5
No. Observations:               10094   AIC:                         1.286e+04
Df Residuals:                   10014   BIC:                         1.344e+04
Df Model:                          79                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 7.3520      0.070    105.721      0.000       7.216       7.488
kellog_price         -4.1813      0.038   -111.348      0.000      -4.255      -4.108
post_price            0.2745      0.030      9.198      0.000       0.216       0.333
quaker_price          0.3090      0.035      8.889      0.000       0.241       0.377
STORE_NUM_387.0       0.5824      0.059      9.822      0.000       0.466       0.699
STORE_NUM_389.0       1.0513      0.056     18.739      0.000       0.941       1.161
STORE_NUM_613.0       0.9140      0.056     16.290      0.000       0.804       1.024
STORE_NUM_623.0       0.7054      0.056     12.570      0.000       0.595       0.815
STORE_NUM_2277.0      1.2726      0.056     22.683      0.000       1.163       1.383
STORE_NUM_2279.0      0.4040      0.056      7.201      0.000       0.294       0.514
STORE_NUM_2281.0      1.0596      0.056     18.885      0.000       0.950       1.170
STORE_NUM_2495.0      0.5910      0.056     10.531      0.000       0.481       0.701
STORE_NUM_2505.0      0.2249      0.056      4.007      0.000       0.115       0.335
STORE_NUM_2513.0      0.8016      0.056     14.285      0.000       0.692       0.912
STORE_NUM_2523.0      0.3858      0.056      6.877      0.000       0.276       0.496
STORE_NUM_2541.0     -0.2922      0.056     -5.208      0.000      -0.402      -0.182
STORE_NUM_4245.0      0.6127      0.056     10.917      0.000       0.503       0.723
STORE_NUM_4259.0      0.9328      0.056     16.625      0.000       0.823       1.043
STORE_NUM_4489.0      0.5276      0.056      9.402      0.000       0.418       0.638
STORE_NUM_4503.0      1.0561      0.056     18.821      0.000       0.946       1.166
STORE_NUM_4521.0     -0.1508      0.056     -2.686      0.007      -0.261      -0.041
STORE_NUM_6179.0      0.7951      0.056     14.170      0.000       0.685       0.905
STORE_NUM_6187.0      0.4858      0.056      8.656      0.000       0.376       0.596
STORE_NUM_6379.0      0.7101      0.056     12.651      0.000       0.600       0.820
STORE_NUM_6431.0      0.2976      0.056      5.303      0.000       0.188       0.408
STORE_NUM_8035.0      0.1559      0.056      2.779      0.005       0.046       0.266
STORE_NUM_8041.0      0.1876      0.056      3.344      0.001       0.078       0.298
STORE_NUM_8263.0      0.3699      0.056      6.592      0.000       0.260       0.480
STORE_NUM_9825.0      1.0254      0.056     18.275      0.000       0.915       1.135
STORE_NUM_10019.0     0.8461      0.056     15.078      0.000       0.736       0.956
STORE_NUM_11757.0     0.6695      0.056     11.932      0.000       0.559       0.779
STORE_NUM_11761.0     0.6816      0.056     12.148      0.000       0.572       0.792
STORE_NUM_11967.0     0.6345      0.056     11.307      0.000       0.525

In [ ]:
# Optimal price = VC * {e/(1+e)}
price = (family_price*0.6)*(-4.1813/(1-4.1813))
price

2.209319882306706

#### Model 3: price variables + store_num + week_end_date 

In [ ]:
# dummy variable
df = pd.get_dummies(data=df,columns=['WEEK_END_DATE'], drop_first = True)

In [ ]:
Y=df['kellog_units']
X = df.drop(['kellog_units','mi_price','private_price'],axis=1)
X = sm.add_constant(X) # adding a constant

model_3 = sm.OLS(Y,X).fit()
model_3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           kellog_units   R-squared:                       0.848
Model:                            OLS   Adj. R-squared:                  0.845
Method:                 Least Squares   F-statistic:                     263.6
Date:                Thu, 03 Jun 2021   Prob (F-statistic):               0.00
Time:                        05:09:48   Log-Likelihood:                -2374.4
No. Observations:               10094   AIC:                             5171.
Df Residuals:                    9883   BIC:                             6694.
Df Model:                         210                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                        6.0584      0.090     67.007      0.000       5.881       6.236
kellog_price                -2.4103      0.056    -43.247      0.000      -2.520      -2.301
post_price                  -0.0825      0.035     -2.333      0.020      -0.152      -0.013
quaker_price                 0.2666      0.059      4.530      0.000       0.151       0.382
STORE_NUM_387.0              0.5531      0.040     13.726      0.000       0.474       0.632
STORE_NUM_389.0              1.0517      0.038     27.614      0.000       0.977       1.126
STORE_NUM_613.0              0.8857      0.038     23.238      0.000       0.811       0.960
STORE_NUM_623.0              0.7772      0.038     20.373      0.000       0.702       0.852
STORE_NUM_2277.0             1.2752      0.038     33.475      0.000       1.200       1.350
STORE_NUM_2279.0             0.4288      0.038     11.255      0.000       0.354       0.503
STORE_NUM_2281.0             1.0749      0.038     28.210      0.000       1.000       1.150
STORE_NUM_2495.0             0.6619      0.038     17.353      0.000       0.587       0.737
STORE_NUM_2505.0             0.3104      0.038      8.132      0.000       0.236       0.385
STORE_NUM_2513.0             0.8655      0.038     22.694      0.000       0.791       0.940
STORE_NUM_2523.0             0.3536      0.038      9.280      0.000       0.279       0.428
STORE_NUM_2541.0            -0.3070      0.038     -8.059      0.000      -0.382      -0.232
STORE_NUM_4245.0             0.5641      0.038     14.779      0.000       0.489       0.639
STORE_NUM_4259.0             0.9371      0.038     24.590      0.000       0.862       1.012
STORE_NUM_4489.0             0.6036      0.038     15.820      0.000       0.529       0.678
STORE_NUM_4503.0             1.0298      0.038     27.011      0.000       0.955       1.105
STORE_NUM_4521.0            -0.0732      0.038     -1.919      0.055      -0.148       0.002
STORE_NUM_6179.0             0.8237      0.038     21.614      0.000       0.749       0.898
STORE_NUM_6187.0             0.5457      0.038     14.310      0.000       0.471       0.620
STORE_NUM_6379.0             0.7897      0.038     20.690      0.000       0.715       0.864
STORE_NUM_6431.0             0.3686      0.038      9.662      0.000       0.294       0.443
STORE_NUM_8035.0             0.1414      0.038      3.712      0.000       0.067       0.216
STORE_NUM_8041.0             0.1973      0.038      5.179      0.000       0.123       0.272
STORE_NUM_8263.0             0.4367      0.038     11.449      0.000       0.362       0.511
STORE_NUM_9825.0             1.0700      0.038     28.079      0.000       0.995       1.145
STORE_NUM_10019.0            0.9149      0.038     23.984      0.000       0.840       0.990
STORE_NUM_1175

In [ ]:
# Optimal price = VC * {e/(1+e)}
price = (family_price*0.6)*(-2.4103/(1-2.4103))
price

2.87284035553134

#### Bias induced by not considering competition

In [ ]:
# regression with No competitors
Y=df['kellog_units']
#X=df[['kellog_price']]
X = df.drop(['kellog_units','mi_price','private_price','post_price','quaker_price'],axis=1)
X = sm.add_constant(X) # adding a constant

model_1_b = sm.OLS(Y,X).fit()
model_1_b.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           kellog_units   R-squared:                       0.848
Model:                            OLS   Adj. R-squared:                  0.845
Method:                 Least Squares   F-statistic:                     265.3
Date:                Thu, 03 Jun 2021   Prob (F-statistic):               0.00
Time:                        05:10:16   Log-Likelihood:                -2387.6
No. Observations:               10094   AIC:                             5193.
Df Residuals:                    9885   BIC:                             6702.
Df Model:                         208                                         
Covariance Type:            nonrobust                                         
============================================================================================
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                        6.1756      0.072     85.247      0.000       6.034       6.318
kellog_price                -2.4180      0.056    -43.365      0.000      -2.527      -2.309
STORE_NUM_387.0              0.5513      0.040     13.670      0.000       0.472       0.630
STORE_NUM_389.0              1.0497      0.038     27.531      0.000       0.975       1.124
STORE_NUM_613.0              0.8834      0.038     23.157      0.000       0.809       0.958
STORE_NUM_623.0              0.7794      0.038     20.412      0.000       0.705       0.854
STORE_NUM_2277.0             1.2730      0.038     33.387      0.000       1.198       1.348
STORE_NUM_2279.0             0.4257      0.038     11.163      0.000       0.351       0.500
STORE_NUM_2281.0             1.0715      0.038     28.102      0.000       0.997       1.146
STORE_NUM_2495.0             0.6616      0.038     17.328      0.000       0.587       0.736
STORE_NUM_2505.0             0.3092      0.038      8.092      0.000       0.234       0.384
STORE_NUM_2513.0             0.8673      0.038     22.723      0.000       0.793       0.942
STORE_NUM_2523.0             0.3551      0.038      9.308      0.000       0.280       0.430
STORE_NUM_2541.0            -0.3066      0.038     -8.039      0.000      -0.381      -0.232
STORE_NUM_4245.0             0.5582      0.038     14.619      0.000       0.483       0.633
STORE_NUM_4259.0             0.9313      0.038     24.425      0.000       0.857       1.006
STORE_NUM_4489.0             0.6046      0.038     15.831      0.000       0.530       0.679
STORE_NUM_4503.0             1.0323      0.038     27.062      0.000       0.958       1.107
STORE_NUM_4521.0            -0.0739      0.038     -1.936      0.053      -0.149       0.001
STORE_NUM_6179.0             0.8183      0.038     21.459      0.000       0.744       0.893
STORE_NUM_6187.0             0.5404      0.038     14.160      0.000       0.466       0.615
STORE_NUM_6379.0             0.7886      0.038     20.653      0.000       0.714       0.863
STORE_NUM_6431.0             0.3677      0.038      9.632      0.000       0.293       0.443
STORE_NUM_8035.0             0.1403      0.038      3.678      0.000       0.066       0.215
STORE_NUM_8041.0             0.1950      0.038      5.114      0.000       0.120       0.270
STORE_NUM_8263.0             0.4383      0.038     11.479      0.000       0.363       0.513
STORE_NUM_9825.0             1.0686      0.038     28.010      0.000       0.994       1.143
STORE_NUM_10019.0            0.9172      0.038     24.015      0.000       0.842       0.992
STORE_NUM_11757.0            0.6996      0.038     18.341      0.000       0.625       0.774
STORE_NUM_11761.0            0.6808      0.038     17.853      0.000       0.606       0.755
STORE_NUM_1196

In [ ]:
with_comp = price
#no_comp = (family_price*0.6)*(-4.1540/(1-4.1540))
no_comp = (family_price*0.6)*(-2.4180/(1-2.4180))
no_comp

2.866368111459869

In [ ]:
bias = with_comp - no_comp
percent_bias = 100*(bias/with_comp)
percent_bias

0.225290766993351